In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data Preprocessing**

### Dangerous Drving Detection Dataset

In [2]:
cd /content/drive/MyDrive/DangerousDrivingRecognition

/content/drive/MyDrive/DangerousDrivingRecognition


In [3]:
!pip install tensorflow_model_optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 238 kB 5.2 MB/s 


In [4]:
from tensorflow import keras
from imutils import paths

from imutils import paths
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os
from time import time
import shutil

from tensorflow.keras.models import load_model
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

In [5]:
# Open the .txt file which have names of training videos
f = open("/content/drive/MyDrive/DangerousDrivingRecognition/traintestlist/trainlist.txt", "r")
temp = f.read()
videos = temp.split('\n')

# Create a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]

In [6]:
# Open the .txt file which have names of test videos
with open("/content/drive/MyDrive/DangerousDrivingRecognition/traintestlist/testlist.txt", "r") as f:
    temp = f.read()
videos = temp.split("\n")

# Create a dataframe having video names
test = pd.DataFrame()
test["video_name"] = videos
test = test[:-1]

In [7]:
def extract_tag(video_path):
    return video_path.split("/")[1].split("_")[0]

def separate_video_name(video_name):
    return video_name.split("/")[1]

def rectify_video_name(video_name):
    return video_name.split(" ")[0]

def move_videos(df, output_dir):
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    for i in tqdm(range(df.shape[0])):
        videoFile = df['video_name'][i].split("/")[-1]
        videoPath = os.path.join("data", videoFile)
        shutil.copy2(videoPath, output_dir)
    print()
    print(f"Total videos: {len(os.listdir(output_dir))}")

train["tag"] = train["video_name"].apply(extract_tag)
train["video_name"] = train["video_name"].apply(separate_video_name)

In [8]:
train["video_name"] = train["video_name"].apply(rectify_video_name)

In [9]:
test["tag"] = test["video_name"].apply(extract_tag)
test["video_name"] = test["video_name"].apply(separate_video_name)

In [10]:
n = 2
topNActs = train["tag"].value_counts().nlargest(n).reset_index()["index"].tolist()
train_new = train[train["tag"].isin(topNActs)]
test_new = test[test["tag"].isin(topNActs)]

In [11]:
train_new = train_new.reset_index(drop=True)
test_new = test_new.reset_index(drop=True)

In [12]:
train_new.to_csv("train.csv", index=False)
test_new.to_csv("test.csv", index=False)

In [13]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

Total videos for training: 472
Total videos for testing: 80


### UCF101

In [34]:
cd /content/drive/MyDrive/DangerousDrivingRecognition/UCF101

/content/drive/MyDrive/DangerousDrivingRecognition/UCF101


In [35]:
!unrar e UCF101.rar data/
!unzip -qq UCF101TrainTestSplits-RecognitionTask.zip

Streaming output truncated to the last 5000 lines.
Extracting  data/v_PlayingFlute_g25_c03.avi                               61%  OK 
Extracting  data/v_PlayingFlute_g25_c04.avi                               61%  OK 
Extracting  data/v_PlayingFlute_g25_c05.avi                               61%  OK 
Extracting  data/v_PlayingFlute_g25_c06.avi                               61%  OK 
Extracting  data/v_PlayingFlute_g25_c07.avi                               61%  OK 
Extracting  data/v_PlayingGuitar_g01_c01.avi                              61%  OK 
Extracting  data/v_PlayingGuitar_g01_c02.avi                              61%  OK 
Extracting  data/v_PlayingGuitar_g01_c03.avi                              61%  OK 
Extracting  data/v_PlayingGuitar_g01_c04.avi                              61%  OK 
Extracting  data/v_PlayingGuitar_g01_c05.avi                              61%  OK 
Extracting  data/v_PlayingGui

In [36]:
!rm -r /content/drive/MyDrive/DangerousDrivingRecognition/UCF101/test
!rm -r /content/drive/MyDrive/DangerousDrivingRecognition/UCF101/train
!rm /content/drive/MyDrive/DangerousDrivingRecognition/UCF101/test.csv
!rm /content/drive/MyDrive/DangerousDrivingRecognition/UCF101/train.csv

rm: cannot remove '/content/drive/MyDrive/DangerousDrivingRecognition/UCF101/test': No such file or directory
rm: cannot remove '/content/drive/MyDrive/DangerousDrivingRecognition/UCF101/train': No such file or directory
rm: cannot remove '/content/drive/MyDrive/DangerousDrivingRecognition/UCF101/test.csv': No such file or directory
rm: cannot remove '/content/drive/MyDrive/DangerousDrivingRecognition/UCF101/train.csv': No such file or directory


In [37]:
!pip install tensorflow_model_optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
from tensorflow import keras
from imutils import paths

from imutils import paths
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os
from time import time
import shutil

from tensorflow.keras.models import load_model
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

In [39]:
# Open the .txt file which have names of training videos
f = open("ucfTrainTestlist/trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# Create a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1


In [40]:
# Open the .txt file which have names of test videos
with open("ucfTrainTestlist/testlist01.txt", "r") as f:
    temp = f.read()
videos = temp.split("\n")

# Create a dataframe having video names
test = pd.DataFrame()
test["video_name"] = videos
test = test[:-1]
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi


In [41]:
def extract_tag(video_path):
    return video_path.split("/")[0]

def separate_video_name(video_name):
    return video_name.split("/")[1]

def rectify_video_name(video_name):
    return video_name.split(" ")[0]

def move_videos(df, output_dir):
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    for i in tqdm(range(df.shape[0])):
        videoFile = df['video_name'][i].split("/")[-1]
        videoPath = os.path.join("data", videoFile)
        shutil.copy2(videoPath, output_dir)
    print()
    print(f"Total videos: {len(os.listdir(output_dir))}")

In [42]:
train["tag"] = train["video_name"].apply(extract_tag)
train["video_name"] = train["video_name"].apply(separate_video_name)
train.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,video_name,tag
0,v_ApplyEyeMakeup_g08_c01.avi 1,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g08_c02.avi 1,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g08_c03.avi 1,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g08_c04.avi 1,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g08_c05.avi 1,ApplyEyeMakeup


In [43]:
train["video_name"] = train["video_name"].apply(rectify_video_name)
train.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,video_name,tag
0,v_ApplyEyeMakeup_g08_c01.avi,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g08_c02.avi,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g08_c03.avi,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g08_c04.avi,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g08_c05.avi,ApplyEyeMakeup


In [44]:
test["tag"] = test["video_name"].apply(extract_tag)
test["video_name"] = test["video_name"].apply(separate_video_name)
test.head()

,video_name,tag
0,v_ApplyEyeMakeup_g01_c01.avi,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g01_c02.avi,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g01_c03.avi,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g01_c04.avi,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g01_c05.avi,ApplyEyeMakeup


In [45]:
n = 101
topNActs = train["tag"].value_counts().nlargest(n).reset_index()["index"].tolist()
train_new = train[train["tag"].isin(topNActs)]
test_new = test[test["tag"].isin(topNActs)]
train_new.shape, test_new.shape

((9537, 2), (3783, 2))

In [46]:
train_new = train_new.reset_index(drop=True)
test_new = test_new.reset_index(drop=True)

In [47]:
move_videos(train_new, "train")
move_videos(test_new, "test")

100%|██████████| 9537/9537 [07:07<00:00, 22.30it/s]



Total videos: 9537


100%|██████████| 3783/3783 [02:54<00:00, 21.68it/s]



Total videos: 3783


In [48]:
train_new.to_csv("train.csv", index=False)
test_new.to_csv("test.csv", index=False)

In [49]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

Total videos for training: 9537
Total videos for testing: 3783


# **Hyperparameters**

In [50]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 500

MAX_SEQ_LENGTH = 30
NUM_FEATURES = 576

# **Models**

In [ ]:
VGG16 = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(160,160,3),
    pooling="avg",
    classes=2,
    classifier_activation="softmax",
)

In [15]:
MobileNet = tf.keras.applications.MobileNet(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling="avg",
    classes=2,
    classifier_activation="softmax",
)

def build_feature_extractor(conv_model):
    
    feature_extractor = conv_model

    preprocess_input = tf.keras.applications.mobilenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="MobileNet")


MobileNet = build_feature_extractor(MobileNet)

17235968/17225924 [==============================] - 0s 0us/step


In [16]:
MobileNetV2 = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling="avg",
    classes=2,
    classifier_activation="softmax",
)

def build_feature_extractor(conv_model):
    
    feature_extractor = conv_model
    
    preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="MobileNetV2")


MobileNetV2 = build_feature_extractor(MobileNetV2)

9420800/9406464 [==============================] - 0s 0us/step


In [17]:
MobileNetV3Small = tf.keras.applications.MobileNetV3Small(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    alpha=1.0,
    minimalistic=False,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    classes=2,
    pooling="avg",
    dropout_rate=0.2,
    classifier_activation="softmax",
    include_preprocessing=True,
)

def build_feature_extractor(conv_model):

    feature_extractor = conv_model

    preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="MobileNetV3Small")


MobileNetV3Small = build_feature_extractor(MobileNetV3Small)

4349952/4334752 [==============================] - 0s 0us/step


In [18]:
NASNetMobile = tf.keras.applications.NASNetMobile(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling="avg",
    classes=2,
)

def build_feature_extractor(conv_model):
    
    feature_extractor = conv_model
    
    preprocess_input = tf.keras.applications.nasnet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="NASNetMobile")


NASNetMobile = build_feature_extractor(NASNetMobile)

20004864/19993432 [==============================] - 0s 0us/step


In [19]:
EfficientNetB0 = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling="avg",
    classes=2,
    classifier_activation="softmax",
)

def build_feature_extractor(conv_model):
    
    feature_extractor = conv_model
    
    preprocess_input = keras.applications.efficientnet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="EfficientNetB0")


EfficientNetB0 = build_feature_extractor(EfficientNetB0)

16719872/16705208 [==============================] - 0s 0us/step


In [20]:
EfficientNetV2B0 = tf.keras.applications.EfficientNetV2B0(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling="avg",
    classes=2,
    classifier_activation="softmax",
    include_preprocessing=True,
)

def build_feature_extractor(conv_model):
    
    feature_extractor = conv_model
    
    preprocess_input = keras.applications.efficientnet_v2.preprocess_input
    
    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="EfficientNetV2B0")


EfficientNetV2B0 = build_feature_extractor(EfficientNetV2B0)

24289280/24274472 [==============================] - 0s 0us/step


In [21]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [22]:
RNN = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH, NUM_FEATURES), name='input'),
    tf.keras.layers.SimpleRNN(16, time_major=False, return_sequences=True),
    tf.keras.layers.SimpleRNN(8),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(n, activation="softmax")
])
RNN.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

GRU = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH, NUM_FEATURES), name='input'),
    tf.keras.layers.GRU(16, time_major=False, return_sequences=True),
    tf.keras.layers.GRU(8),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(n, activation="softmax")
])
GRU.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

LSTM = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH, NUM_FEATURES), name='input'),
    tf.keras.layers.LSTM(16, time_major=False, return_sequences=True),
    tf.keras.layers.LSTM(8),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(n, activation="softmax")
])
LSTM.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


LSTM.summary()
GRU.summary()
RNN.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 16)            37952     
                                                                 
 lstm_1 (LSTM)               (None, 8)                 800       
                                                                 
 dropout_2 (Dropout)         (None, 8)                 0         
                                                                 
 flatten_2 (Flatten)         (None, 8)                 0         
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 2)                 18        
                                                                 
Total params: 38,842
Trainable params: 38,842
Non-trai

In [ ]:
LSTM2 = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH, NUM_FEATURES), name='input'),
    tf.keras.layers.LSTM(40, time_major=False, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(160, activation="relu"),
    # tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(n, activation="softmax"),
])
adam = tf.keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
rms = tf.keras.optimizers.RMSprop()
LSTM2.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])
LSTM2.summary()

# **Training**

In [51]:
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [52]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)

In [ ]:
def prepare_all_videos(df, root_dir, feature_extractor):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        print(f'{idx}/{len(video_paths)}')
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):

                # interpreter = tf.lite.Interpreter(model_path="MobileNet.tflite")
                # interpreter.allocate_tensors()
                # input_details = interpreter.get_input_details()
                # output_details = interpreter.get_output_details()
                # input_shape = input_details[0]['shape']
                # input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
                # input_data[0] = batch[None, j, :]
                # interpreter.set_tensor(input_details[0]['index'], input_data)
                # interpreter.invoke()
                # output_data = interpreter.get_tensor(output_details[0]['index'])
                # temp_frame_features[i, j, :] = output_data

                temp_frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train", MobileNetV3Small)
test_data, test_labels = prepare_all_videos(test_df, "test", MobileNetV3Small)

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

0/9537
1/9537
2/9537
3/9537
4/9537
5/9537
6/9537
7/9537
8/9537
9/9537
10/9537
11/9537
12/9537
13/9537
14/9537
15/9537
16/9537
17/9537
18/9537
19/9537
20/9537
21/9537
22/9537
23/9537
24/9537
25/9537
26/9537
27/9537
28/9537
29/9537
30/9537
31/9537
32/9537
33/9537
34/9537
35/9537
36/9537
37/9537
38/9537
39/9537
40/9537
41/9537
42/9537
43/9537
44/9537
45/9537
46/9537
47/9537
48/9537
49/9537
50/9537
51/9537
52/9537
53/9537
54/9537
55/9537
56/9537
57/9537
58/9537
59/9537
60/9537
61/9537
62/9537
63/9537
64/9537
65/9537
66/9537
67/9537
68/9537
69/9537
70/9537
71/9537
72/9537
73/9537
74/9537
75/9537
76/9537
77/9537
78/9537
79/9537
80/9537
81/9537
82/9537
83/9537
84/9537
85/9537
86/9537
87/9537
88/9537
89/9537
90/9537
91/9537
92/9537
93/9537
94/9537
95/9537
96/9537
97/9537
98/9537
99/9537
100/9537
101/9537
102/9537
103/9537
104/9537
105/9537
106/9537
107/9537
108/9537
109/9537
110/9537
111/9537
112/9537
113/9537
114/9537
115/9537
116/9537
117/9537
118/9537
119/9537
120/9537
121/9537
122/9537
123

In [ ]:
# Utility for running experiments.
def run_experiment(seq_model):
    filepath = "/content/drive/MyDrive/DangerousDrivingRecognition/DangerousDrivingRecognition"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    history = seq_model.fit(
        [train_data[0]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
        use_multiprocessing=True,
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model

print("RNN")
history_rnn, RNN = run_experiment(RNN)
print("\n")
print("GRU")
history_gru, GRU = run_experiment(GRU)
print("\n")
print("LSTM")
history_lstm, LSTM = run_experiment(LSTM)

## **Saving Models**

In [ ]:
cd /content

In [ ]:
MobileNet.save("MobileNet.h5")
MobileNetV2.save("MobileNetV2.h5")
MobileNetV3Small.save("MobileNetV3Small.h5")
NASNetMobile.save("NASNetMobile.h5")
EfficientNetB0.save("EfficientNetB0.h5")
EfficientNetV2B0.save("EfficientNetV2B0.h5")
RNN.save("RNN.h5")
GRU.save("GRU.h5")
LSTM.save("LSTM.h5")

# **TF Model**

### Backbones

In [ ]:
run_model = tf.function(lambda x: MobileNet(x))
# This is important, let's fix the input size.
BATCH_SIZE = BATCH_SIZE
STEPS = MAX_SEQ_LENGTH
INPUT_SIZE = NUM_FEATURES
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([None, 224, 224, 3], MobileNet.inputs[0].dtype))
# model directory.
MODEL_DIR_MobileNet = "keras_MobileNet"
MobileNet.save(MODEL_DIR_MobileNet, save_format="tf", signatures=concrete_func)


run_model = tf.function(lambda x: MobileNetV2(x))
# This is important, let's fix the input size.
BATCH_SIZE = BATCH_SIZE
STEPS = MAX_SEQ_LENGTH
INPUT_SIZE = NUM_FEATURES
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([None, 224, 224, 3], MobileNetV2.inputs[0].dtype))
# model directory.
MODEL_DIR_MobileNetV2 = "keras_MobileNetV2"
MobileNetV2.save(MODEL_DIR_MobileNetV2, save_format="tf", signatures=concrete_func)


run_model = tf.function(lambda x: MobileNetV3Small(x))
# This is important, let's fix the input size.
BATCH_SIZE = BATCH_SIZE
STEPS = MAX_SEQ_LENGTH
INPUT_SIZE = NUM_FEATURES
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([None, 224, 224, 3], MobileNetV3Small.inputs[0].dtype))
# model directory.
MODEL_DIR_MobileNetV3Small = "keras_MobileNetV3Small"
MobileNetV3Small.save(MODEL_DIR_MobileNetV3Small, save_format="tf", signatures=concrete_func)


run_model = tf.function(lambda x: EfficientNetB0(x))
# This is important, let's fix the input size.
BATCH_SIZE = BATCH_SIZE
STEPS = MAX_SEQ_LENGTH
INPUT_SIZE = NUM_FEATURES
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([None, 224, 224, 3], EfficientNetB0.inputs[0].dtype))
# model directory.
MODEL_DIR_EfficientNetB0 = "keras_EfficientNetB0"
EfficientNetB0.save(MODEL_DIR_EfficientNetB0, save_format="tf", signatures=concrete_func)


run_model = tf.function(lambda x: EfficientNetV2B0(x))
# This is important, let's fix the input size.
BATCH_SIZE = BATCH_SIZE
STEPS = MAX_SEQ_LENGTH
INPUT_SIZE = NUM_FEATURES
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([None, 224, 224, 3], EfficientNetV2B0.inputs[0].dtype))
# model directory.
MODEL_DIR_EfficientNetV2B0 = "keras_EfficientNetV2B0"
EfficientNetV2B0.save(MODEL_DIR_EfficientNetV2B0, save_format="tf", signatures=concrete_func)


run_model = tf.function(lambda x: NASNetMobile(x))
# This is important, let's fix the input size.
BATCH_SIZE = BATCH_SIZE
STEPS = MAX_SEQ_LENGTH
INPUT_SIZE = NUM_FEATURES
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([None, 224, 224, 3], NASNetMobile.inputs[0].dtype))
# model directory.
MODEL_DIR_NASNetMobile = "keras_NASNetMobile"
NASNetMobile.save(MODEL_DIR_NASNetMobile, save_format="tf", signatures=concrete_func)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_MobileNet)
tflite_model = converter.convert()
with open('MobileNet.tflite', 'wb') as f:
  f.write(tflite_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_MobileNetV2)
tflite_model = converter.convert()
with open('MobileNetV2.tflite', 'wb') as f:
  f.write(tflite_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_MobileNetV3Small)
tflite_model = converter.convert()
with open('MobileNetV3Small.tflite', 'wb') as f:
  f.write(tflite_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_EfficientNetB0)
tflite_model = converter.convert()
with open('EfficientNetB0.tflite', 'wb') as f:
  f.write(tflite_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_EfficientNetV2B0)
tflite_model = converter.convert()
with open('EfficientNetV2B0.tflite', 'wb') as f:
  f.write(tflite_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_NASNetMobile)
tflite_model = converter.convert()
with open('NASNetMobile.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_MobileNet)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()
with open('MobileNet_quantized.tflite', 'wb') as f:
  f.write(tflite_quant_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_MobileNetV2)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()
with open('MobileNetV2_quantized.tflite', 'wb') as f:
  f.write(tflite_quant_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_MobileNetV3Small)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()
with open('MobileNetV3Small_quantized.tflite', 'wb') as f:
  f.write(tflite_quant_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_EfficientNetB0)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()
with open('EfficientNetB0_quantized.tflite', 'wb') as f:
  f.write(tflite_quant_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_EfficientNetV2B0)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()
with open('EfficientNetV2B0_quantized.tflite', 'wb') as f:
  f.write(tflite_quant_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_NASNetMobile)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()
with open('NASNetMobile_quantized.tflite', 'wb') as f:
  f.write(tflite_quant_model)

### Sequence Models

In [ ]:
run_model = tf.function(lambda x: RNN(x))
# This is important, let's fix the input size.
BATCH_SIZE = BATCH_SIZE
STEPS = MAX_SEQ_LENGTH
INPUT_SIZE = NUM_FEATURES
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], RNN.inputs[0].dtype))
# model directory.
MODEL_DIR_RNN = "keras_rnn"
RNN.save(MODEL_DIR_RNN, save_format="tf", signatures=concrete_func)


run_model = tf.function(lambda x: GRU(x))
# This is important, let's fix the input size.
BATCH_SIZE = BATCH_SIZE
STEPS = MAX_SEQ_LENGTH
INPUT_SIZE = NUM_FEATURES
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], GRU.inputs[0].dtype))
# model directory.
MODEL_DIR_GRU = "keras_gru"
GRU.save(MODEL_DIR_GRU, save_format="tf", signatures=concrete_func)


run_model = tf.function(lambda x: LSTM(x))
# This is important, let's fix the input size.
BATCH_SIZE = BATCH_SIZE
STEPS = MAX_SEQ_LENGTH
INPUT_SIZE = NUM_FEATURES
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], LSTM.inputs[0].dtype))
# model directory.
MODEL_DIR_LSTM = "keras_lstm"
LSTM.save(MODEL_DIR_LSTM, save_format="tf", signatures=concrete_func)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_RNN)
tflite_model = converter.convert()

with open('RNN.tflite', 'wb') as f:
  f.write(tflite_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_GRU)
tflite_model = converter.convert()

with open('GRU.tflite', 'wb') as f:
  f.write(tflite_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_LSTM)
tflite_model = converter.convert()

with open('LSTM.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_RNN)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()
with open('RNN_quantized.tflite', 'wb') as f:
  f.write(tflite_quant_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_GRU)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()
with open('GRU_quantized.tflite', 'wb') as f:
  f.write(tflite_quant_model)


converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR_LSTM)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()
with open('LSTM_quantized.tflite', 'wb') as f:
  f.write(tflite_quant_model)

# **Inference Time & Accuracy**

### Keras Inference

In [ ]:
def prepare_single_video(frames, feature_extractor):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")
    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked
    return frame_features, frame_mask


def sequence_prediction(path, feature_extractor, sequence_model):
    class_vocab = label_processor.get_vocabulary()
    frames = load_video(os.path.join("test", path))
    start = time()
    frame_features, frame_mask = prepare_single_video(frames, feature_extractor)
    probabilities = sequence_model.predict([frame_features, ])[0]
    end = time()
    inference_time = end - start
    return inference_time, frames

In [ ]:
def with_opencv(filename):
    video = cv2.VideoCapture(filename)

    duration = video.get(cv2.CAP_PROP_POS_MSEC)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)

    return duration, frame_count


timing_rnn = 0
timing_gru = 0
timing_lstm = 0
for i in range(20):
  test_video = np.random.choice(test_df["video_name"].values.tolist())
  inference_time_rnn, test_frames = sequence_prediction(test_video, NASNetMobile, RNN)
  inference_time_gru, test_frames = sequence_prediction(test_video, NASNetMobile, GRU)
  inference_time_lstm, test_frames = sequence_prediction(test_video, NASNetMobile, LSTM)
  filename = f"/content/drive/MyDrive/DangerousDrivingRecognition/test/{test_video}"
  duration, frame_count = with_opencv(filename)
  timing_rnn += ((inference_time_rnn) / frame_count) * 1000
  timing_gru += ((inference_time_gru) / frame_count) * 1000
  timing_lstm += ((inference_time_lstm) / frame_count) * 1000
timing_rnn /= 20
timing_gru /= 20
timing_lstm /= 20
print(f"\n\n\n\nRNN Processing of each frame: {timing_rnn}\n")
print(f"GRU Processing of each frame: {timing_gru}\n")
print(f"LSTM Processing of each frame: {timing_lstm}\n\n")


_, accuracy = RNN.evaluate([test_data[0], ], test_labels)
print(f"RNN Test Accuracy: {round(accuracy * 100, 2)}%\n")
_, accuracy = GRU.evaluate([test_data[0], ], test_labels)
print(f"GRU Test Accuracy: {round(accuracy * 100, 2)}%\n")
_, accuracy = LSTM.evaluate([test_data[0], ], test_labels)
print(f"LSTM Test Accuracy: {round(accuracy * 100, 2)}%\n")

ValueError: ignored

### TF Inference

In [ ]:
def with_opencv(filename):
    video = cv2.VideoCapture(filename)

    duration = video.get(cv2.CAP_PROP_POS_MSEC)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)

    return duration, frame_count


inference_time_rnn = 0
for i in range(20):
  test_video = np.random.choice(test_df["video_name"].values.tolist())

  class_vocab = label_processor.get_vocabulary()

  frames = load_video(os.path.join("test", test_video))
  start = time()

  frames = frames[None, ...]
  frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

  for i, batch in enumerate(frames):
      video_length = batch.shape[0]
      length = min(MAX_SEQ_LENGTH, video_length)
      for j in range(length):
          interpreter = tf.lite.Interpreter(model_path="MobileNet_quantized.tflite")
          interpreter.allocate_tensors()
          input_details = interpreter.get_input_details()
          output_details = interpreter.get_output_details()
          input_shape = input_details[0]['shape']
          input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
          input_data[0] = batch[None, j, :]
          interpreter.set_tensor(input_details[0]['index'], input_data)
          interpreter.invoke()
          output_data = interpreter.get_tensor(output_details[0]['index'])
          frame_features[i, j, :] = output_data


  interpreter = tf.lite.Interpreter(model_path="RNN_quantized.tflite")
  interpreter.allocate_tensors()
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  input_shape = input_details[0]['shape']
  input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
  input_data[0] = frame_features
  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])
  probabilities = output_data

  end = time()
  inference_time_rnn = end - start


  filename = f"/content/drive/MyDrive/DangerousDrivingRecognition/test/{test_video}"
  duration, frame_count = with_opencv(filename)
  
  inference_time_rnn += ((inference_time_rnn) / frame_count) * 1000


inference_time_rnn /= 20

print(f"\n\n\n\nRNN Processing of each frame: {inference_time_rnn}\n")






RNN Processing of each frame: 30.013563157121343

